# Naïve Bayes Classifier

***Примечание 1.*** В основе NBC (Naïve Bayes Classifier) для классификации лежит, **теорема Байеса** :
\\[P(H|E) = \frac{P(H) * P(E|H)}{P(E)}\\] 

Для принятия решения нам потребуется научиться вычислять следующие величины:
\\[P(спам|сообщение) = \frac{P(спам) * P(сообщение|спам)}{P(сообщение)}\\]
где,
* \\(P(спам|сообщение)\\) - вероятность что сообщение принадлежит классу cпам, именно её нам надо рассчитать;
* \\(P(спам)\\) - безусловная вероятность встретить сообщение класса спам в корпусе сообщений;
* \\(P(сообщение|спам)\\) - вероятность встретить сообщение среди всех собщений класса спам;
* \\(P(сообщение)\\) - безусловная вероятность сообщения в корпусе соообщений.
  
***Аналогично рассматривается величина \\(P(не\spaceспам|сообщение)\\)***

***Примечание 2.*** Цель классификации состоит в том чтобы понять к какому классу принадлежит сообщение, поэтому нам нужна не сама вероятность, а наиболее вероятный класс(спам / не спам).
Байесовский классификатор использует оценку **апостериорного максимума (Maximum a posteriori estimation)** для определения наиболее вероятного класса. Грубо говоря, это класс с максимальной вероятностью.

\\[C_{\text{map}} = max(\frac{P(спам) * P(сообщение|спам)}{P(сообщение)}, \frac{P(не\spaceспам) * P(сообщение|не\spaceспам)}{P(сообщение)})\\]

То есть нам надо рассчитать вероятность для всех классов и выбрать тот класс, который обладает максимальной вероятностью.

***Примечание 3.*** знаменатель (вероятность сообщения) является константой и никак не может повлиять на ранжирование классов, поэтому в нашей задаче мы можем его игнорировать.
**Формула №1:**
\\[C_{\text{map}} = max(\space P(спам) * P(сообщение|спам), P(не\spaceспам) * P(сообщение|не\spaceспам)\space)\\]

***Примечание 4.*** Байесовский же классификатор представляет документ как набор слов вероятности которых условно не зависят друг от друга.

Исходя из этого предположения условная вероятность документа аппроксимируется произведением условных вероятностей всех слов входящих в документ.

*например для сообщения "Купите наш товар!" принадлежещего классу **Спам***:
\\[P(Купите\spaceнаш\spaceтовар|спам) ≈ P(Купите|спам) * P(наш|спам) * P(товар|спам)\\] 

Подставив полученное выражение в общем виде в **Формула №1** мы получим:
\\[C_{\text{map}} = max(\space P(спам) * \prod_{i=1}^{n}P(слово\space из\space сообщения|спам), P(не\spaceспам) * \prod_{i=1}^{n}P(слово\space из\space сообщения|не\spaceспам)\space)\\]

***Примечание 5.***  При достаточно большой длине документа придется перемножать большое количество очень маленьких чисел. Для того чтобы при этом избежать арифметического переполнения снизу зачастую пользуются свойством логарифма произведения: \\(log(ab) = \log(a) + \log(b)\\)

\\[C_{\text{map}} = max(\space \log P(спам) + \sum_{i=1}^{n}\log P(слово\space из\space сообщения|спам), \log P(не\spaceспам) + \sum_{i=1}^{n}\log P(слово\space из\space сообщения|не\spaceспам)\space)\\]

***Примечание 6.*** **Проблема неизвестных слов :**
Но так как может оказаться даже что отдельное слово больше нигде не встречалось, то следует пойти на шаг дальше и применить так называемое сглаживание Лапласа, при котором мы для каждого слова мы притворяемся, что у нас есть ещё два письма: одно содержащее это слово и одно без такого слова. Т.е. финальная формула для вычисления
такой вероятности выглядит так:

\\[P(слово|спам) = \frac{P(спам-писем\space с\space словом + 1)}{P(спам-писем + 2)}\\]

# Реализация

In [217]:
import pandas as pd
import numpy as np
from collections import defaultdict
import os
import nltk

In [218]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Ilya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Сбор данных: Соберем набор писем, которые уже помечены как спам или не спам. Создадим DataFrame на основе каталога содержащего наборы данных Enron-Spam.

In [219]:
def read_files(directory, label):
    """
    Прочитывает все файлы из directory и подготавливает их к упаковке в DF
    :param directory: директория из которой будут читаться файлы
    :param label: метка показывающая класс файла (spam/nam)
    :return: возвращает список подготовленных для упаковки в DF сообщений
    *(сообщение готово к упаковке в DF, если:
        -все слова находятся в lowerCase
        -отсутствуют знаки пунктуации, в т.ч. \space
        -сообщение разбито на токены ([word1, word2, .... wordn])
    """
    messages = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            text = preprocessing(file.read())
            messages.append((label, text))
    return messages


def create_dataFrame(messages):
    """
    Возвращает dataFrame с подготовленными данными для обучения классификатора на основе Naïve Bayes Classifier
    :param messages: Preprocessed Data (подготовленные данные)
    :return: dataFrame
    """
    df = pd.DataFrame(messages, columns=['Class', 'Message'])
    return df


def preprocessing(text):
    """
    Функция необходима для переработки text.
    Функция выполняет:
        1) Очистка text - удаление ненужных символов, таких как пунктуация, специальные символы.
        2) Приведение к нижнему регистру
        3) Токенизация text - разделение text на отдельные слова или токены.

    :param text: текст, который будет переработан
    :return: переработанный text
    """
    # Перевод слов в нижний регистр
    text = text.lower()

    # Удаление знаков пунткуации
    punctuation = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    translator = str.maketrans('', '', punctuation)
    text = text.translate(translator)

    #Токенизация
    words = text.split()
    words = list(set(words))

    return words


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Ilya\AppData\Local\Temp\ipykernel_12624\2304485705.py:2: SyntaxWarning: invalid escape sequence '\s'
  """


In [232]:
path_directory_spam = r"\Naive Bayes Classifier\data\spam"
path_directory_ham = r"\Naive Bayes Classifier\data\ham"

messages_spam = read_files(path_directory_spam, "spam")
messages_ham = read_files(path_directory_ham, "ham")

all_messages = messages_ham + messages_spam

df = create_dataFrame(all_messages)
df

,Class,Message
0,ham,"[am, 2, to, eott, need, analysts, i, business,..."
1,ham,"[am, frusco, here, changes, means, minns, clea..."
2,ham,"[help, entex, attached, let, can, it, to, prov..."
3,ham,"[continental, result, point, injury, ian, revi..."
4,ham,"[tim, rather, functionality, am, cc, were, res..."
...,...,...
4134,spam,"[few, 49, to, professionai, languages, like, n..."
4135,spam,"[few, 49, to, professional, inciuding, languag..."
4136,spam,"[rest, leads, panic, is, hate, fear, it, 2005,..."
4137,spam,"[one, uk, is, all, 2005, when, just, microsoft..."


### Найдем оценку вероятности встретить каждое слово в спамном или не спамном сообщении. 

Посчитаем сколько раз каждое слово встречается в каждом из типов сообщений:

In [233]:
def counts_word(data):
    """
    создает словарь вида {key = слово; value = [#количество вхождений в сообщения spam, #количество вхождений в сообщения ham]
    :param data: dataFrame с указанием класса сообщения (spam/ham) и токенизированным сообщением
    :return: словарь вида {key = слово; value = [#количество вхождений в сообщения spam, #количество вхождений в сообщения ham]
    """
    counts = defaultdict(lambda: [0,0])
    for idx, row in data.iterrows():
        for word in row["Message"]:
            counts[word][0 if  row["Class"] =='spam' else 1] +=1
    return counts

In [234]:
freq = counts_word(df)
freq

defaultdict(<function __main__.counts_word.<locals>.<lambda>()>,
            {'am': [246, 422],
             '2': [509, 263],
             'to': [2374, 1029],
             'eott': [0, 5],
             'need': [406, 328],
             'analysts': [10, 36],
             'i': [840, 722],
             'business': [685, 258],
             'me': [284, 528],
             'analysis': [24, 59],
             'roles': [2, 29],
             'commercial': [37, 101],
             'months': [117, 78],
             'a': [2102, 866],
             'related': [55, 63],
             'of': [2031, 831],
             '6': [290, 108],
             'currently': [97, 106],
             'work': [260, 201],
             'subject': [2939, 1200],
             'in': [1732, 770],
             'period': [39, 30],
             'for': [1860, 915],
             'frusco': [0, 2],
             'here': [944, 179],
             'changes': [132, 119],
             'means': [144, 20],
             'minns': [0, 2],
            

Сохраним в переменные общее число сообщений класса spam и ham :

In [235]:
all_spam = df['Class'].loc[df['Class'] == 'spam'].count()
all_ham = df['Class'].loc[df['Class'] == 'ham'].count()

all_spam, all_ham

(np.int64(2939), np.int64(1200))

Сохраним в переменные Априорные вероятности \\(P(спам)\\) \\(P(не \space спам)\\) :

In [236]:
p_spam = all_spam / (all_spam + all_ham)
p_ham = all_ham / (all_spam + all_ham)

p_spam, p_ham

(np.float64(0.7100748973181928), np.float64(0.2899251026818072))

Для каждого слова найдем вероятность встретить его в каждом из классов сообщений spam, hum :
***(Найдем \\(P(слово|не\spaceспам)\\) и \\(P(слово|спам)\\))*** Используя сглаживание Лапласса.

In [237]:
words_prob = {}
for word, counts in freq.items():
    spam_count, ham_count = counts
    prob_word_in_spam = (spam_count + 1) / (all_spam + 2)
    prob__word_in_ham = (ham_count + 1) / (all_ham + 2)
    words_prob[word] = (prob_word_in_spam, prob__word_in_ham)


In [238]:
words_prob

{'am': (np.float64(0.08398503910234614), np.float64(0.3519134775374376)),
 '2': (np.float64(0.17341040462427745), np.float64(0.21963394342762063)),
 'to': (np.float64(0.8075484529071745), np.float64(0.8569051580698835)),
 'eott': (np.float64(0.00034002040122407346),
  np.float64(0.004991680532445923)),
 'need': (np.float64(0.1383883032981979), np.float64(0.2737104825291181)),
 'analysts': (np.float64(0.003740224413464808),
  np.float64(0.030782029950083195)),
 'i': (np.float64(0.2859571574294458), np.float64(0.6014975041597338)),
 'business': (np.float64(0.23325399523971438),
  np.float64(0.21547420965058237)),
 'me': (np.float64(0.09690581434886093), np.float64(0.4400998336106489)),
 'analysis': (np.float64(0.008500510030601836),
  np.float64(0.04991680532445923)),
 'roles': (np.float64(0.0010200612036722204),
  np.float64(0.024958402662229616)),
 'commercial': (np.float64(0.01292077524651479),
  np.float64(0.08485856905158069)),
 'months': (np.float64(0.04012240734444067), np.float64

### Определим к какому классу (ham/ spam) относится сообщение

In [239]:
def classify_message(words_prob, message, p_spam, p_ham):
    """
    Функция классифицирует message и определяет к какому классу оно относится 
    :param words_prob: словарь вида {key = слово; value = [P(вхождения в сообщения spam), P(вхождения в сообщения ham)]
    :param message: сообщение для классификации
    :param p_spam: Априорная вероятность, сообщение - спам
    :param p_ham: Априорная вероятность, сообщение - не спам
    :return:
    """
    message_words = preprocessing(message)
    log_prob_spam = np.log(p_spam)
    log_prob_ham = np.log(p_ham)
    for word in message_words:
        if word in words_prob:
            prob_word_in_spam, prob_word_in_ham = words_prob[word]
            log_prob_spam += np.log(prob_word_in_spam)
            log_prob_ham += np.log(prob_word_in_ham)

    if log_prob_spam > log_prob_ham :
        return 'spam'
    return 'ham'

### Узнаем точность классификации

In [240]:
numb_right_classified = 0; # счетчик точно классифицированных писем
count_emails = 0

In [244]:
def testing(directory,words_prob, p_spam, p_ham, class_email):
    global numb_right_classified
    global count_emails
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            res = classify_message(words_prob, file.read(), p_spam, p_ham)
            if res == class_email : numb_right_classified+=1
            count_emails+=1
    return 

In [245]:
dir_spam_test = r"\Naive Bayes Classifier\data\spam_test"
dir_ham_test = r"\Naive Bayes Classifier\data\ham_test"

In [246]:
testing(dir_spam_test, words_prob, p_spam, p_ham, 'spam')
testing(dir_ham_test, words_prob, p_spam, p_ham, 'ham')

print(numb_right_classified/ count_emails)

0.9575289575289575
